In [1]:
import pyvista as pv
import numpy as np
np.random.seed(0)

import forward_growth
import inverse_growth
import mesh_laplacian

## Fast-growing sphere destroys patterns

In [ ]:
# import mesh
filename = 'inputs/sphere.ply'
mesh = pv.read(filename)

mesh = mesh.triangulate()
mesh = mesh.decimate(0.7)
mesh = mesh.triangulate()

# integration params
nx = mesh.n_points
niter = 1000
dt = 0.0001
dx = .002/2 # good for decimated sphere

# RD params
du = 1
dv = 10
g = 1000 # set to zero to just test diffusion
a = 0.126779*0.25
b = 1.1

# filenames
pts_file = 'examples/pts_static_sphere.npy'
laps_file = 'examples/laps_static_sphere.npy'
output_gif = 'examples/static_sphere.gif'


In [ ]:
# grow sphere
forward_growth.grow_forward(mesh, nx, niter, dt, growth_rate=1/dt, grow=True, grow_from_rule=True, pts_output=pts_file, laps_output=laps_file)

In [ ]:
# integrate RD
laps = np.load(laps_file)
u_stored, v_stored = forward_growth.rd_forward(du, dv, g, a, b, nx, dx, niter, dt, laps)

In [ ]:
# plot result
pts = np.load(pts_file)
forward_growth.plot_forward(mesh, u_stored, pts, niter, nskip=2, output_gif=output_gif)


## Static sphere allows pattern to form

In [ ]:
# import mesh
filename = 'inputs/sphere.ply'
mesh = pv.read(filename)

mesh = mesh.triangulate()
mesh = mesh.decimate(0.7)
mesh = mesh.triangulate()

# integration params
nx = mesh.n_points
niter = 10
dt = 0.0001
dx = .002/2 # good for decimated sphere

# RD params
du = 1
dv = 10
g = 1000 # set to zero to just test diffusion
a = 0.126779*0.25
b = 1.1

# filenames
pts_file = 'examples/pts_static_sphere.npy'
laps_file = 'examples/laps_static_sphere.npy'
output_gif = 'examples/static_sphere.gif'


In [ ]:
# grow sphere
forward_growth.grow_forward(mesh, nx, niter, dt, growth_rate=.01/dt, grow=False, grow_from_rule=True, pts_output=pts_file, laps_output=laps_file)

In [ ]:
# integrate RD
laps = np.load(laps_file)
u_stored, v_stored = forward_growth.rd_forward(du, dv, g, a, b, nx, dx, niter, dt, laps)

In [ ]:
# plot result
pts = np.load(pts_file)
forward_growth.plot_forward(mesh, u_stored, pts, niter, nskip=2, output_gif=output_gif)


## Slow-growing sphere -> Different pattern depending on growth speed

In [ ]:
# import mesh
filename = 'inputs/sphere.ply'
mesh = pv.read(filename)

mesh = mesh.triangulate()
mesh = mesh.decimate(0.7)
mesh = mesh.triangulate()

# integration params
nx = mesh.n_points
niter = 6000
dt = 0.0001
dx = .002/2 # good for decimated sphere

# RD params
du = 1
dv = 10
g = 1000 # set to zero to just test diffusion
a = 0.126779*0.25
b = 1.1

# filenames
data_file = 'examples/data/slow_sphere_3.npz'
mesh_file = 'examples/data/slow_sphere_3.vtk'
output_gif = 'examples/slow_sphere_3.gif'


# grow sphere
# growth_rate = 0.0005/dt
# forward_growth.grow_forward(mesh, nx, niter, dt, growth_rate=growth_rate, grow=True, grow_from_rule=True, output_file=data_file)

# integrate RD
# don't really need to recalc laps when growth is isotropic, just do it easily by scaling here

growth_factors = np.linspace(0.0, 0.1, niter)

# Initialize points
scale_factor = 1
dx /= scale_factor
pts = np.zeros((nx, 3, niter+1))
mesh.points *= scale_factor
pts[:,:,0] = mesh.points.copy()


# Initialize laps
laps = np.zeros((nx, nx, niter+1))
laps[:,:,0] = mesh_laplacian.compute_mesh_laplacian(mesh)

# calculate pts and laps via growth factors
for i in range(niter):
    pts[:,:,i+1] = pts[:,:,0] * (1 + growth_factors[i])
    laps[:,:,i] = laps[:,:,0] #/ (1 + growth_factors[i])**2 # can rescale dx to eliminate this effect

# write results to disk
np.savez_compressed(data_file, pts=pts, laps=laps)

import sys
def rd_forward(mesh, pts, du, dv, g, a, b, nx, dx, niter, dt, laps, grow=False, output_file=None):
    '''
        Integrates RD DE's forward in time on growing manifold defined by laps.
    '''
    from forward import step_se

    # initialize fields near steady-state solution
    u = np.ones(nx, dtype=float)*(a+b)
    u += np.random.normal(scale=0.01, size=nx)
    v = np.ones(nx, dtype=float)*(b/(a+b)**2)

    u_stored = np.zeros((nx, niter+1))
    u_stored[:,0] = u

    v_stored = np.zeros((nx, niter+1))
    v_stored[:,0] = v


    integrate = True

    if integrate:
        print("Beginning RD integration loop...")
        for i in range(niter):
            sys.stdout.write("\rIteration {0}/{1} ({2}%)".format(i+1, niter, int(100*(i+1)/niter)))
            sys.stdout.flush()

            # Run GMRES to solve for next timestep
            # reference calculated laplacians from growth loop
            if grow: 
                # first apply dilution
                area_current = mesh.area
                mesh.points  = pts[:,:,i+1]
                area_new     = mesh.area
                dilution_factor = area_current / area_new
                
                # dilution_factor = 0.99 # just to test
                # print(dilution_factor)
                
                u *= dilution_factor
                v *= dilution_factor
                
                
                # then RD
                u, v = step_se(u,v, a,b,g,du,dv, laps[:,:,i], dx,nx,dt)
            else: u, v = step_se(u,v, a,b,g,du,dv, laps, dx,nx,dt)

            # store for later animation
            u_stored[:,i+1] = u
            v_stored[:,i+1] = v


        print("\nRD loop completed.")
        
    # write results to disk
    # note that this overwrites pts and laps
    # np.savez_compressed(output_file, u=u_stored, v=v_stored)
    
    print("RD data written to {}.".format(output_file))
    
    # reset mesh
    if grow: mesh.points = pts[:,:,0]
    
    
    return u_stored, v_stored

laps[0,0,0] / dx**2

In [ ]:
u_stored, v_stored = rd_forward(mesh, pts, du, dv, g, a, b, nx, dx, niter, dt, laps, grow=True, output_file=data_file)

In [ ]:
# write solution onto mesh
mesh.point_data['u'] = u_stored[:,-1]
mesh.point_data['v'] = v_stored[:,-1]

# save mesh to disk
mesh.save(mesh_file)

mesh.points = pts[:,:,-1]
print(mesh.area)

In [ ]:
# plot result
pts = np.load(data_file)['pts']
# u_stored = np.zeros((mesh.n_points, niter))
params = [du, dv, g, a, b, nx, dx, niter, dt]

forward_growth.plot_forward(params, mesh, u_stored, pts, niter, nskip=120, output_gif=output_gif, mode="static", grow=True)


In [ ]:
growth_rate = 0.0005/dt

(1 + growth_rate * dt * niter)**2

In [ ]:
print(pts[:,0])
print(pts[:,1])
print(pts[:,2])

## Spots/stripes on static cow

In [22]:
# import mesh
filename = 'inputs/cow.vtp'
mesh = pv.read(filename)

mesh = mesh.triangulate()
mesh = mesh.decimate(0.2)
mesh = mesh.triangulate()

# integration params
nx = mesh.n_points
niter = 1500
dt = 0.0001
dx = .2/5 # good for decimated cow

# RD params
du = 1
dv = 10
g = 1000 # set to zero to just test diffusion
a = 0.126779*0.25
b = 0.792366 # spots, 3
# b = 1.1 # stripes, 2
# b = 1.5 # uniform, 1

# filenames
data_file = 'examples/data/cow_4.npz'
mesh_file = 'examples/data/cow_4.vtk'
output_gif = 'examples/cow_4.gif'


In [23]:
# grow sphere
# forward_growth.grow_forward(mesh, nx, niter, dt, growth_rate=1/dt, grow=False, grow_from_rule=True, output_file=data_file)


In [24]:
# integrate RD
laps = mesh_laplacian.compute_mesh_laplacian(mesh)
u_stored, v_stored = forward_growth.rd_forward(du, dv, g, a, b, nx, dx, niter, dt, laps, output_file=data_file)

# write solution onto mesh
mesh.point_data['u'] = u_stored[:,-1]
mesh.point_data['v'] = v_stored[:,-1]

# save mesh to disk
mesh.save(mesh_file)

Beginning RD integration loop...
Iteration 1500/1500 (100%)
RD loop completed.


In [26]:
# plot result    
pts = mesh.points
params = [du, dv, g, a, b, nx, dx, niter, dt]
forward_growth.plot_forward(params, mesh, u_stored, pts, niter, nskip=25, output_gif=output_gif, mode='static', cpos='xy', rot=-45)


Beginning plotting loop...
Iteration 1500/1500 (100%)
Plotting loop completed.
Plotting completed.


## Spots/stripes on static sphere

In [ ]:
# import mesh
filename = 'inputs/sphere.ply'
mesh = pv.read(filename)

mesh = mesh.triangulate()
mesh = mesh.decimate(0.5)
mesh = mesh.triangulate()

# integration params
nx = mesh.n_points
niter = 3000
dt = 0.0001
dx = .002/2 # good for decimated sphere

# RD params
du = 1
dv = 10
g = 1000 # set to zero to just test diffusion
a = 0.126779*0.25
b = 1.5 # uniform
# b = 1.1 # stripes
# b = .79 # spots


# filenames
data_file = 'examples/data/sphere_3.npz'
mesh_file = 'examples/data/sphere_3.vtk'
output_gif = 'examples/sphere_3.gif'


In [ ]:
# grow sphere
# forward_growth.grow_forward(mesh, nx, niter, dt, growth_rate=1/dt, grow=False, grow_from_rule=True, output_file=data_file)


In [ ]:
# integrate RD
laps = mesh_laplacian.compute_mesh_laplacian(mesh)
u_stored, v_stored = forward_growth.rd_forward(du, dv, g, a, b, nx, dx, niter, dt, laps, output_file=data_file)

# write solution onto mesh
mesh.point_data['u'] = u_stored[:,-1]
mesh.point_data['v'] = v_stored[:,-1]

# save mesh
# save mesh to disk
mesh.save(mesh_file)

In [ ]:
# plot result    
pts = mesh.points
params = [du, dv, g, a, b, nx, dx, niter, dt]
forward_growth.plot_forward(params, mesh, u_stored, pts, niter, nskip=30, output_gif=output_gif, mode='dynamic', cpos='xy', rot=-45)


## Equal diffusion coefficients

In [ ]:
# import mesh
filename = 'inputs/sphere.ply'
mesh = pv.read(filename)

mesh = mesh.triangulate()
mesh = mesh.decimate(0.7)
mesh = mesh.triangulate()

# integration params
nx = mesh.n_points
niter = 450
dt = 0.0001
dx = .002/2 # good for decimated sphere
# dx = .2 # good for decimated cow

# RD params
du = 1
dv = 1
g = 1000 # set to zero to just test diffusion
a = 0.126779*0.25
b = 0.792366

# filenames
data_file = 'examples/data/sphere_equal.npz'
output_gif = 'examples/sphere_equal.gif'


In [ ]:
# grow 
forward_growth.grow_forward(mesh, nx, niter, dt, growth_rate=1/dt, grow=False, grow_from_rule=True, output_file=data_file)


In [ ]:
# integrate RD

laps = np.load(data_file)['laps']
u_stored, v_stored = forward_growth.rd_forward(du, dv, g, a, b, nx, dx, niter, dt, laps)

In [ ]:
# plot result
pts = np.load(data_file)['pts']
forward_growth.plot_forward(mesh, u_stored, pts, niter, nskip=100, output_gif=output_gif, mode='static')


Very interesting: I'm seeing pattern formation with equal diffusion coefficients? Could curvature be making that possible? Should revisit this.

Figured out why: it was just that I accidentally hard-coded the RD parameters into forward_growth.py. After deleting them the behavior is as expected: equal diffusion constants yield no pattern.

## Curvature coupling

In [2]:
# import mesh
filename = 'inputs/sphere.ply'
mesh = pv.read(filename)

mesh = mesh.triangulate()
mesh = mesh.decimate(0.5)
mesh = mesh.triangulate()
nx = mesh.n_points

# stretch points along z for ellipse
# mesh.points[:,2] *= 2

# print(mesh.points[:,2].max())
# flattens top of sphere
for i in range(nx):
    z_plane = 0
    if mesh.points[i,2] > z_plane:
        # mesh.points[i,2] = z_plane - mesh.points[i,2] # pop inward
        mesh.points[i,2] = z_plane # flatten

# integration params
nx = mesh.n_points
niter = 1200
dt = 0.0001
dx = .001 # good for decimated sphere
# dx = .2 # good for decimated cow

# RD params - coupled to curvature
# du = np.ones(nx)*1/5
# dv = np.ones(nx)*10/5
curv =  mesh.curvature('gaussian')
du = 0.2 + 1 * np.array(abs(curv)<1e-16, dtype=float)
dv = du.copy() * 10
g = 1000 # set to zero to just test diffusion
a = 0.126779*0.25
b = 0.792366

# filenames
data_file = 'examples/data/sphere_coupled.npz'
output_gif = 'examples/sphere_coupled.gif'
               
               
# pl = pv.Plotter()
# _ = pl.add_mesh(mesh, show_edges=True, scalars=mesh.curvature("gaussian"))
# _ = pl.add_axes_at_origin(ylabel=None)
# pl.camera_position = 'xz'
# pl.show()


In [3]:
# grow 
forward_growth.grow_forward(mesh, nx, niter, dt, growth_rate=1/dt, grow=False, grow_from_rule=True, output_file=data_file)


Growth loop completed.
Growth data written to examples/data/sphere_coupled.npz.


In [4]:
# integrate RD
laps = np.load(data_file)['laps']
u_stored, v_stored = forward_growth.rd_forward(du, dv, g, a, b, nx, dx, niter, dt, laps, grow=True, )

Beginning RD integration loop...
Iteration 1200/1200 (100%)
RD loop completed.


In [9]:
# plot result
pts = np.load(data_file)['pts']
params = [du, dv, g, a, b, nx, dx, niter, dt]

forward_growth.plot_forward(params, mesh, u_stored, pts, niter, nskip=10, output_gif=output_gif, mode='static', cpos='zx')


Beginning plotting loop...
Iteration 1200/1200 (100%)
Plotting loop completed.
Plotting completed.


In [ ]:
du